In [1]:
from openpyxl import load_workbook, Workbook
from openpyxl.styles import colors
from openpyxl.styles import Font, Color, PatternFill, Alignment, Border, Side
from datetime import time, datetime, date, timedelta
from openpyxl.utils import get_column_letter

In [31]:
hours_wb = load_workbook(filename = '2023_Spring_Semester_Schedule1.xlsx')
hours_sheet = hours_wb['Sheet1']

header = Font(name='Calibri', size=20, bold=True, italic=False, vertAlign=None, underline='none', strike=False, color='00000000')
days = Font(name='Calibri', size=15, bold=True, italic=False, vertAlign=None, underline='none', strike=False, color='00000000')
cell_aligment = Alignment(horizontal='center', vertical='center', text_rotation=0, wrap_text=False, shrink_to_fit=False, indent=0)
time_borders = Border(left=Side(border_style='medium', color='00C0C0C0'), right=Side(border_style='medium', color='00C0C0C0'),  bottom=Side(border_style='medium', color='00C0C0C0'))
rectangle_border = Border(left=Side(border_style='thick', color='00000000'), right=Side(border_style='thick', color='00000000'),  bottom=Side(border_style='thick', color='00000000'), top=Side(border_style='thick', color='00000000'))

new_wb = Workbook()
new_sheet = new_wb.active
hours_col_a = hours_sheet['A']

new_sheet['A1'] = 'Diging Schedule'
new_sheet['A1'].font = header

new_sheet.column_dimensions["A"].width = 24
new_sheet.column_dimensions["B"].width = 8

for x in range(3, 35):
    new_sheet.column_dimensions[get_column_letter(x)].width = 4

row_number = 3

days_of_weeks_strings = ['Monday', 'Tuesady', 'Wednesday', 'Thursday', 'Friday']

background_colors = ["FFBA84", '90B44B', '7DB9DE', '787D7B', '005CAF', 'EFBB24', '994639', '1B813E', '52433D', '81C7D4', '8E354A', '554236', 'F7D94C', '646A58', '7B90D2']

worker_map = {}

for day in days_of_weeks_strings:
    new_sheet.cell(row=row_number, column=1, value=day).font = days
    count = 0
    start_column = 3
    end_column = start_column + 3
    time_list = ['9:00', '10:00', '11:00', '12:00', '1:00', '2:00', '3:00', '4:00', '5:00']
    while count < 8:
        new_sheet.cell(row=row_number, column=start_column, value=time_list[count]).alignment = cell_aligment
        new_sheet.cell(row=row_number, column=start_column, value=time_list[count]).border = time_borders
        new_sheet.merge_cells(start_row=row_number, start_column=start_column, end_row=row_number, end_column=end_column)
        start_column = end_column + 1
        end_column = start_column + 3
        count += 1
    row_number += 1
    for index, name in enumerate(hours_col_a):
        if index != 0:
            new_sheet['A' + str(row_number)] = name.value
            if name.value in worker_map:
                worker_map[name.value].append(row_number)
            else:
                worker_map[name.value] = [row_number]
            row_number += 1
    row_number += 1

for index, row in enumerate(hours_sheet.iter_rows()):
    if index != 0:
        name = row[0].value
        max_avail_minutes = int(row[1].value) * 60
        scheduled_minutes = 0
        row_numbers = worker_map[name]
        day = 1
        start = 2
        end = 3
        while day < 6:
            #change var name not called string
            start_string = row[start].value
            end_string = row[end].value
            cols_to_add = start_string.hour - 9
            
            start_column = 3 + (cols_to_add * 5)
            print(start_column)
            if isinstance(start_string, time):
                duration = datetime.combine(date.min, end_string) - datetime.combine(date.min, start_string)
                total_cells = duration.seconds / 60 / 15
                print("Total Cells: {}".format(total_cells))
                # current times go one cell past everytime
                total_cells = total_cells - 1
                temp_scheduled_minutes = scheduled_minutes
                temp_scheduled_minutes += duration.seconds / 60
                if temp_scheduled_minutes <= max_avail_minutes:
                    end_column = start_column + total_cells
                    schedule_cell = new_sheet.cell(row=row_numbers[day-1], column=start_column)
                    schedule_cell.fill = PatternFill("solid", fgColor=background_colors[index])
                    schedule_cell.border = rectangle_border 
                    new_sheet.merge_cells(start_row=row_numbers[day-1], start_column=start_column, end_row=row_numbers[day-1], end_column=end_column)
                    schedule_cell.value = datetime.strptime(start_string.isoformat(timespec='minutes'), "%H:%M").strftime("%I:%M %p") +' - ' + datetime.strptime(end_string.isoformat(timespec='minutes'), "%H:%M").strftime("%I:%M %p")
                    new_sheet.cell(row=row_numbers[day-1], column=start_column).alignment = cell_aligment
                    start = start + 2
                    end = end + 2
                    scheduled_minutes = temp_scheduled_minutes
                else:
                    left_mins = max_avail_minutes - scheduled_minutes
                    if left_mins >= 120:
                        dt = datetime.combine(date.today(), start_string) + timedelta(minutes=left_mins)
                        total_cells = left_mins / 15
                        total_cells = total_cells - 1
                        end_column = start_column + total_cells
                        m1 = 'C' + str(row_numbers[day-1]) 
                        new_sheet[m1].fill = PatternFill("solid", fgColor=background_colors[index])
                        new_sheet[m1].border = rectangle_border 
                        new_sheet.merge_cells(start_row=row_numbers[day-1], start_column=start_column, end_row=row_numbers[day-1], end_column=end_column)
                        new_sheet[m1] = datetime.strptime(start_string.isoformat(timespec='minutes'), "%H:%M").strftime("%I:%M %p") +' - ' + datetime.strptime(dt.time().isoformat(timespec='minutes'), "%H:%M").strftime("%I:%M %p")
                        new_sheet[m1].alignment = cell_aligment
                        start = start + 2
                        end = end + 2
                        scheduled_minutes = temp_scheduled_minutes
            day += 1



new_wb.save(filename = 'new_schedule.xlsx')

3
Total Cells: 30.0


AttributeError: 'NoneType' object has no attribute 'hour'

In [34]:
{'dotted', 'dashDot', 'thin', 'hair', 'mediumDashDotDot', 'slantDashDot', 'double', 'medium', 'mediumDashDot', 'dashed', 'thick', 'mediumDashed', 'dashDotDot'}

False

In [2]:
hours_wb = load_workbook(filename = 'hours.xlsx')
hours_sheet = hours_wb['Sheet1']

In [5]:
for index, row in enumerate(hours_sheet.iter_rows()):
    if index != 0:
        print(row.value)

AttributeError: 'tuple' object has no attribute 'value'

In [10]:
row[2].value - row[3].value

TypeError: unsupported operand type(s) for -: 'datetime.time' and 'datetime.time'

In [2]:
hours_wb = load_workbook(filename = 'hours2.xlsx')
hours_sheet = hours_wb['Sheet1']

header = Font(name='Calibri', size=20, bold=True, italic=False, vertAlign=None, underline='none', strike=False, color='00000000')
days = Font(name='Calibri', size=15, bold=True, italic=False, vertAlign=None, underline='none', strike=False, color='00000000')
cell_aligment = Alignment(horizontal='center', vertical='center', text_rotation=0, wrap_text=False, shrink_to_fit=False, indent=0)
time_borders = Border(left=Side(border_style='medium', color='00C0C0C0'), right=Side(border_style='medium', color='00C0C0C0'),  bottom=Side(border_style='medium', color='00C0C0C0'))
rectangle_border = Border(left=Side(border_style='thick', color='00000000'), right=Side(border_style='thick', color='00000000'),  bottom=Side(border_style='thick', color='00000000'), top=Side(border_style='thick', color='00000000'))

new_wb = Workbook()
new_sheet = new_wb.active
hours_col_a = hours_sheet['A']

new_sheet['A1'] = 'Diging Schedule'
new_sheet['A1'].font = header

new_sheet.column_dimensions["A"].width = 24
new_sheet.column_dimensions["B"].width = 8

for x in range(3, 35):
    new_sheet.column_dimensions[get_column_letter(x)].width = 4

row_number = 3

days_of_weeks_strings = ['Monday', 'Tuesady', 'Wednesday', 'Thursday', 'Friday']

background_colors = ["FFBA84", '90B44B', '7DB9DE', '787D7B', '005CAF', 'EFBB24', '994639', '1B813E', '52433D', '81C7D4', '8E354A', '554236', 'F7D94C', '646A58', '7B90D2']

worker_map = {}

for day in days_of_weeks_strings:
    new_sheet.cell(row=row_number, column=1, value=day).font = days
    count = 0
    start_column = 3
    end_column = start_column + 3
    # lab stats -  should be changed to be dynamic
    lab_open_time = time(hour=9)
    lab_close_time = time(hour=17)
    lab_duration = datetime.combine(date.min, lab_close_time) - datetime.combine(date.min, lab_open_time)
    # total number of 15 minute increments in lab duration - equates to total # of cells available
    total_quarter_increments = divmod((lab_duration.total_seconds()/60), 15)
    print(f"Quarter: {total_quarter_increments}")
    time_list = ['9:00', '10:00', '11:00', '12:00', '1:00', '2:00', '3:00', '4:00', '5:00']
    while count < 8:
        new_sheet.cell(row=row_number, column=start_column, value=time_list[count]).alignment = cell_aligment
        new_sheet.cell(row=row_number, column=start_column, value=time_list[count]).border = time_borders
        new_sheet.merge_cells(start_row=row_number, start_column=start_column, end_row=row_number, end_column=end_column)
        start_column = end_column + 1
        end_column = start_column + 3
        count += 1
    row_number += 1
    for index, name in enumerate(hours_col_a):
        if index != 0:
            new_sheet['A' + str(row_number)] = name.value
            if name.value in worker_map:
                worker_map[name.value].append(row_number)
            else:
                worker_map[name.value] = [row_number]
            row_number += 1
    row_number += 1

for index, row in enumerate(hours_sheet.iter_rows()):
    if index != 0:
        # check if there is a name, if not consider it the end of the list
        if not row[0].value:
            break
        name = row[0].value
        max_avail_minutes = int(row[1].value) * 60
        scheduled_minutes = 0
        row_numbers = worker_map[name]
        day = 1
        start = 2
        end = 3
        while day < 6:
            print(f"******************Day: {day}*****************")
            start_string = row[start].value
            if isinstance(start_string, time):
                #change var name not called string - its a time object
                
                end_string = row[end].value
                print(f"Start String: {start_string}, End String: {end_string}")
                start_string_2 = row[start + 2].value
                end_string_2 = row[end + 2].value
                print(f"start 2: {start_string_2}, end 2: {end_string_2}")
                cols_to_add = start_string.hour - 9

                start_column = 3 + (cols_to_add * 5)
                print(f"Start Column: {start_column}")
                # Get durations of both days in hours
                duration = datetime.combine(date.min, end_string) - datetime.combine(date.min, start_string)
                duration_quarter_increments = divmod((duration.total_seconds()/60), 15)
                total_cells = total_cells - 1
                end_column = (start_column + duration_quarter_increments[0]) - 1
                schedule_cell = new_sheet.cell(row=row_numbers[day-1], column=start_column)
                schedule_cell.fill = PatternFill("solid", fgColor=background_colors[index])
                schedule_cell.border = rectangle_border 
                new_sheet.merge_cells(start_row=row_numbers[day-1], start_column=start_column, end_row=row_numbers[day-1], end_column=end_column)
                cell_string = datetime.strptime(start_string.isoformat(timespec='minutes'), "%H:%M").strftime("%I:%M %p") +' - ' + datetime.strptime(end_string.isoformat(timespec='minutes'), "%H:%M").strftime("%I:%M %p")
                if (duration.seconds / 60) > 300:
                    print('Duration: {}'.format(duration.seconds))
                    cell_string = cell_string + ' (1/2 hour lunch)'
                schedule_cell.value = cell_string
                new_sheet.cell(row=row_numbers[day-1], column=start_column).alignment = cell_aligment
                print(f"Duration 1: {duration}, Quarter: {day_quarter_increments}")
                if isinstance(start_string_2, time):
                    duration_2 = datetime.combine(date.min, end_string_2) - datetime.combine(date.min, start_string_2)
                    break_duration = datetime.combine(date.min, start_string_2) - datetime.combine(date.min, end_string)
                    break_quarter_increments = divmod((break_duration.total_seconds()/60), 15)
                    duration_2_quarter_increments = divmod((break_duration.total_seconds()/60), 15)
                    # only start column is off by 1
                    print(f"Incoming Start: {start_column}, end column: {end_column}")
                    start_column = (end_column + break_quarter_increments[0]) + 1
                    end_column = (start_column + duration_2_quarter_increments[0]) - 1
                    print(f"After Start: {start_column}, end column: {end_column}")
                    schedule_cell = new_sheet.cell(row=row_numbers[day-1], column=start_column)
                    schedule_cell.fill = PatternFill("solid", fgColor=background_colors[index])
                    schedule_cell.border = rectangle_border 
                    new_sheet.merge_cells(start_row=row_numbers[day-1], start_column=start_column, end_row=row_numbers[day-1], end_column=end_column)
                    cell_string = datetime.strptime(start_string.isoformat(timespec='minutes'), "%H:%M").strftime("%I:%M %p") +' - ' + datetime.strptime(end_string.isoformat(timespec='minutes'), "%H:%M").strftime("%I:%M %p")
                    if (duration.seconds / 60) > 300:
                        print('Duration: {}'.format(duration.seconds))
                        cell_string = cell_string + ' (1/2 hour lunch)'
                    schedule_cell.value = cell_string
                    new_sheet.cell(row=row_numbers[day-1], column=start_column).alignment = cell_aligment
                    print(f"Duration 2: {duration_2}, break duration: {break_duration}")
            else:
                print('OFF')

            start = start + 4
            end = end + 4

            day += 1



new_wb.save(filename = 'new_schedule.xlsx')

Quarter: (32.0, 0.0)
Quarter: (32.0, 0.0)
Quarter: (32.0, 0.0)
Quarter: (32.0, 0.0)
Quarter: (32.0, 0.0)
******************Day: 1*****************
Start String: 09:00:00, End String: 16:30:00
start 2: None, end 2: None
Start Column: 3


NameError: name 'total_cells' is not defined

In [6]:
490 / 15

32.666666666666664

In [8]:
a = divmod(490, 15)
a[1]

10

In [29]:
new_sheet.max_row

71